

---



# **Get the code and models**

In [1]:
#@title <h1>Install Wav2Lip</h1>
#@markdown * Install dependencies
#@markdown * Download models
!rm -rf sample_data
!mkdir sample_data

In [ ]:
!git clone https://github.com/zabique/Wav2Lip

#download the pretrained model
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O 'Wav2Lip/checkpoints/wav2lip_gan.pth'
!wget 'https://iiitaphyd-my.sharepoint.com/:u:/g/personal/radrabha_m_research_iiit_ac_in/Eb3LEzbfuKlJiR600lQWRxgBIY27JZg80f7V9jtMfbNDaQ?e=TBFBVW' -O 'Wav2Lip/checkpoints/wav2lip.pth'
!pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

# !pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip && pip install -r requirements.txt

#download pretrained model for face detection
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "Wav2Lip/face_detection/detection/sfd/s3fd.pth"

!pip install -q youtube-dl
!pip install ffmpeg-python


#this code for recording audio
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
# from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

# %cd /
# from ghc.l_ghc_cf import l_ghc_cf
# %cd content

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr


from IPython.display import HTML
from base64 import b64encode
def showVideo(path):
  mp4 = open(str(path),'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width=700 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url)

from IPython.display import clear_output 
# clear_output()
print("\nDone")



---



# **Quick guide**
1. Create video file input_video.mp4
2. Create audio file input_audio.wav
3. Both files have to be exact same length
4. Target face in the input_video.mp4, must be in ALL videoframes (So no black or blurry frames etc)


# **Now lets try!**

In [ ]:
#@title 1.Upload input_video.mp4 & input_audio.wav files
# %cd sample_data/
# from google.colab import files
# uploaded = files.upload()
# %cd ..

In [ ]:
pip install pydub


In [ ]:
from pydub import AudioSegment

def convert_mp3_to_wav(mp3_file, wav_file):
    # Load the MP3 file
    audio = AudioSegment.from_mp3("/kaggle/input/video-dataset/audio.mp3")

    # Export as WAV
    audio.export(wav_file, format="wav")

# Example usage
convert_mp3_to_wav("input.mp3", "output.wav")



In [ ]:
convert_mp3_to_wav("input.mp3", "/kaggle/working/output.wav")


In [ ]:
#@title 1a. Record your own voice
# from IPython.display import Audio 
# from IPython.core.display import display

# record_or_upload = 'record' #@param ['record', 'Upload']

# def displayAudio():
#   display(Audio('/content/sample_data/input_audio.wav'))
# if record_or_upload == 'record':
#   audio, sr = get_audio()
#   import scipy
#   scipy.io.wavfile.write('/content/sample_data/input_audio.wav', sr, audio)
# elif record_or_upload == 'Upload':
#   from google.colab import files
#   uploaded = files.upload()
#   for fn in uploaded.keys():
#     print('User uploaded file "{name}" with length {length} bytes'.format(
#         name=fn, length=len(uploaded[fn])))
  
#   #concider only the first file
#   audio_file = str(list(uploaded.keys())[0])
#   !rm -f '/content/sample_data/input_audio.wav'
#   !ffmpeg -i '$audio_file' '/content/sample_data/input_audio.wav'

#   clear_output()
#   displayAudio()

In [ ]:
#@title 2.Create Wav2Lip video (using wav2lip_gan.pth) GAN
%cd Wav2Lip

!python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/kaggle/input/wav2lip-dataset/input_video.mp4" --audio "/kaggle/input/wav2lip-dataset/input_audio.wav"
# !python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/kaggle/input/wav2lip-dataset/MonnaLisa.mp4" --audio "/kaggle/input/wav2lip-dataset/MonnaLisa.wav"
# !python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/kaggle/input/wav2lip-dataset/girl.mp4" --audio "/kaggle/input/wav2lip-dataset/girl.wav"

# !python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/kaggle/input/wav2lip-dataset/MonnaLisa.mp4" --audio "/kaggle/input/wav2lip-dataset/AI.wav"
# !python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/kaggle/input/wav2lip-dataset/girl.mp4" --audio "/kaggle/input/wav2lip-dataset/AI2.wav"

# from IPython.display import clear_output 
# clear_output()
# print("\nDone! now press X")

In [ ]:
#@title 3.Play result video -  50% scaling
from IPython.display import HTML
from base64 import b64encode
mp4 = open('results/result_voice.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="50%" height="50%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

In [ ]:
# #@title 4.Download Result.mp4 to your computer
# from google.colab import files
# files.download('/content/Wav2Lip/results/result_voice.mp4') 


In [ ]:
# #@title 5. Delete old result files
# %rm /content/sample_data/*
# from IPython.display import clear_output 
# clear_output()
# print("\nDone! now press X")

In [ ]:
# #@title 1-4. Batch processing - Upload -> process -> download -> play result
# %cd sample_data/
# %rm input_audio.wav
# %rm input_video.mp4
# from google.colab import files
# uploaded = files.upload()
# %cd ..
# !cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/sample_data/input_video.mp4" --audio "/content/sample_data/input_audio.wav"
# from google.colab import files
# files.download('/content/Wav2Lip/results/result_voice.mp4') 
# from IPython.display import HTML
# from base64 import b64encode
# mp4 = open('/content/Wav2Lip/results/result_voice.mp4','rb').read()
# data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
# HTML(f"""
# <video width="50%" height="50%" controls>
#       <source src="{data_url}" type="video/mp4">
# </video>""")

# **Variations to try**


In [ ]:
# #@title 2.Use resize_factor to reduce the video resolution, as there is a change you might get better results for lower resolution videos. Why? Because the model was trained on low resolution faces.
# !cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/sample_data/input_video.mp4" --audio "/content/sample_data/input_audio.wav" --resize_factor 2

In [ ]:
# #@title 3.Use more padding to include the chin region (u can manually edit pads dimensions viewing and changing the code)
# !cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/sample_data/input_video.mp4" --audio "/content/sample_data/input_audio.wav" --pads 0 20 0 0

In [ ]:
# #@title 4.Play result video -  50% scaling
# from IPython.display import HTML
# from base64 import b64encode
# mp4 = open('/content/Wav2Lip/results/result_voice.mp4','rb').read()
# data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
# HTML(f"""
# <video width="50%" height="50%" controls>
#       <source src="{data_url}" type="video/mp4">
# </video>""")

In [ ]:
# #@title 5.Download Result.mp4 to your computer
# from google.colab import files
# files.download('/content/Wav2Lip/results/result_voice.mp4') 
